Scoring in next 10 actions model - Model Evaluation - GBM

In [ ]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

from exp_vaep.exp_vaep_config import exp_vaep_modelling_v2_file_path
from exp_vaep.domain.contracts.modelling_data_contract import ModellingDataContract
from exp_vaep.domain.modelling.supermodel import SuperXGBRegressor
from exp_vaep.domain.modelling.model_evaluation import XGBRegressorEvaluator
from exp_vaep.domain.preprocessing.preprocessing import *

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

New Model

In [ ]:
new_version = 2
model_name = "exp_vaep_scoring"
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp_vaep/model_outputs/models/"
preproc_file_path = ""

Comparison Model

In [ ]:
old_version = new_version - 1
compare_train_predictions_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp_vaep/model_outputs/predictions/train_predictions_"+model_name+"_v"+str(old_version)+".csv"
compare_test_predictions_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp_vaep/model_outputs/predictions/test_predictions_"+model_name+"_v"+str(old_version)+".csv"

Response

In [ ]:
RESPONSE = ModellingDataContract.RESPONSE_SCORES

Features

In [ ]:
FEATURES = ModellingDataContract.feature_list_scores

Load Data

In [ ]:
modelling_data = pd.read_csv(exp_vaep_modelling_v2_file_path)
modelling_data.tail()

Preprocess Data

In [ ]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [ ]:
training_data = modelling_data[modelling_data[RESPONSE+"TrainingSet"]]
test_data = modelling_data[modelling_data[RESPONSE+"TestSet"]]
cal_data = modelling_data[modelling_data[RESPONSE+"ValidationSet"]]

In [ ]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [ ]:
y_train.mean(), y_test.mean(), y_cal.mean()

In [ ]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

Load Model

In [ ]:
super_xgb = joblib.load(model_file_path + "/" + model_name + "_v" + str(new_version) + ".joblib")

Get Predictions

In [ ]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

Compare Model

In [ ]:
# compare_train_data = pd.read_csv(compare_train_predictions_file_path)
# compare_test_data = pd.read_csv(compare_test_predictions_file_path)

In [ ]:
# compare_train_data = compare_train_data.rename(columns = {'xgb_preds_exp_scores':'old_preds'})
# compare_test_data = compare_test_data.rename(columns = {'xgb_preds_exp_scores':'old_preds'})

In [ ]:
# training_data.shape, compare_train_data.shape, test_data.shape, compare_test_data.shape

In [ ]:
# compare_test_data.head()

In [ ]:
# training_data = pd.merge(training_data, compare_train_data[["match_id", "quarter", "quarter_seconds", "overall_seconds", "team", "player", 'start_x', 'start_y', 'end_x', 'end_y', "action_type", "outcome_type", "xScore", 'old_preds']], how = "left", on = ["match_id", "quarter", "quarter_seconds", "overall_seconds", "team", "player", 'start_x', 'start_y', 'end_x', 'end_y', "action_type", "outcome_type", "xScore",])
# test_data = pd.merge(test_data, compare_test_data[["match_id", "quarter", "quarter_seconds", "overall_seconds", "team", "player", 'start_x', 'start_y', 'end_x', 'end_y', "action_type", "outcome_type", "xScore", 'old_preds']], how = "left", on = ["match_id", "quarter", "quarter_seconds", "overall_seconds", "team", "player", 'start_x', 'start_y', 'end_x', 'end_y', "action_type", "outcome_type", "xScore",])

Evaluate Model

In [ ]:
training_data['xgb_preds'] = train_preds

train_xgb_evals = XGBRegressorEvaluator(model = super_xgb,
                                       data = training_data,
                                       actual_name = RESPONSE,
                                       expected_name = "xgb_preds",
                                    #    compare_name='old_preds',
                                       )


In [ ]:
test_data['xgb_preds'] = test_preds

test_xgb_evals = XGBRegressorEvaluator(model = super_xgb,
                                       data = test_data,
                                       actual_name = RESPONSE,
                                       expected_name = "xgb_preds",
                                    #    compare_name = "old_preds",
                                       )

Model Averages

In [ ]:
training_data[[RESPONSE, 'xgb_preds']].describe()

In [ ]:
test_data[[RESPONSE, 'xgb_preds']].describe()

Plots

In [ ]:
train_xgb_evals.plot_distribution()

In [ ]:
test_xgb_evals.plot_distribution()

In [ ]:
train_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")

In [ ]:
train_xgb_evals.plot_shap_summary_plot(sample=1000)

In [ ]:
test_xgb_evals.plot_shap_summary_plot(sample=10000)

In [ ]:
top_features = test_xgb_evals.get_ranked_feature_importance()
top_10_features = top_features[:10]
top_10_features

In [ ]:
# for col in top_10_features:
#     test_xgb_evals.plot_pdp([col])

In [ ]:
# for col in top_10_features:
#     train_xgb_evals.plot_ice([col])
#     test_xgb_evals.plot_ice([col])

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_feature_ave(col)
    test_xgb_evals.plot_feature_ave(col)

Model Metrics (for probabilities, mainly want calibration curve, logloss and brier score loss)

In [ ]:
train_xgb_evals.get_mae(), test_xgb_evals.get_mae()

In [ ]:
train_xgb_evals.get_mse(), test_xgb_evals.get_mse() 

In [ ]:
train_xgb_evals.get_r2_score(), test_xgb_evals.get_r2_score() 